In [1]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from evaluate import load
from datasets import load_dataset, Audio

In [19]:
TEXT_COLUMN_NAME = 'text'
SPLIT_NAME = 'validation'
MODEL_DIR = r"D:\WhiteLake\University\Diploma\Practice\MyStep2Model\server_files\models\checkpoint-libri-clean"
DATASET_DIR = r"D:\WhiteLake\University\Diploma\Practice\Datasets\!final_versions\augmented-librispeech-ru\folder"

In [5]:
cer_metric = load("cer")

Model part

In [6]:
model = Wav2Vec2ForCTC.from_pretrained(MODEL_DIR).cuda()

In [7]:
processor = Wav2Vec2Processor.from_pretrained(MODEL_DIR)

In [8]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch[TEXT_COLUMN_NAME] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [9]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch[TEXT_COLUMN_NAME]).input_ids
    return batch

Dataset part

In [20]:
test_dataset = load_dataset("parquet", data_dir=DATASET_DIR, split=SPLIT_NAME)

In [21]:
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [22]:
test_dataset

Dataset({
    features: ['text', 'audio'],
    num_rows: 1400
})

In [23]:
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)
results = test_dataset.map(map_to_result, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [24]:
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=results["pred_str"], references=results[TEXT_COLUMN_NAME])))

Test CER: 0.070


In [30]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    
show_random_elements(results)

,pred_str,text
0,зачем крутить ся ветра вовраге подъемлет лист и пыль несет когда корамбля в ней движные влаге его дыхание же одно ждет,зачем крутится ветр в овраге подъемлет лист и пыль несет когда корабль в недвижной влаге его дыханья жадно ждет
1,зачем судьбой не суждено моей непостоянной лире геройство воспевать одно и с ним не знае мое в мире любовь и дружбу старых лет,зачем судьбой не суждено моей непостоянной лире геройство воспевать одно и с ним незнаемые в мире любовь и дружбу старых лет
2,там в блаках перед народом через леса через моря колдун несет богатыря,там в облаках перед народом через леса через моря колдун несет богатыря
3,ясниця вещи сон герою он видит будто бы княжна над страшной безно голубиною стоит недвижно и бледна,и снится вещий сон герою он видит будто бы княжна над страшной бездны глубиною стоит недвижна и бледна
4,красуйся град петров истой неколебимо как россия да умерится же с тобой и побежденная стихия,красуйся град петров и стой неколебимо как россия да умирится же с тобой и побежденная стихия
5,все тихо страх его объемлет по нем текут и жар и хлат,все тихо страх его объемлет по нем текут и жар и хлад
6,раскинув не в от поволнам рыбак навесло наклоненный плывет колесистым берегам к порогу хижины смиренной,раскинув невод по волнам рыбак на весла наклоненный плывет к лесистым берегам к порогу хижины смиренной
7,его чело его ланиты мгновенным пламенем горят,его чело его ланиты мгновенным пламенем горят
8,я каждый день востав отсна благодарю сеждечно бога зато что в наше времена волшебников не так уж много к тому же чести славы им женить бы наши безопасны,я каждый день восстав от сна благодарю сердечно бога за то что в наши времена волшебников не так уж много к тому же честь и слава им женитьбы наши безопасны
9,казак пловет начал маке глоча под нуричному сети,казак плывет на челноке влача по дну речному сети
